In [1]:
from IPython.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [32]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
from desdeo_emo.EAs import RVEA, NSGAIII, IBEA
from desdeo_problem.testproblems.TestProblems import test_problem_builder
from desdeo_problem import DataProblem
#from desdeo_problem.testproblems.DBMOPP import DBMOPP
from desdeo_tools.utilities import fast_non_dominated_sort, hypervolume_indicator

import matplotlib.pyplot as plt
import sklearn
from pyDOE import lhs
import pandas as pd

In [3]:
#pip install pyDOE

In [4]:
    elif design == "LHSDesign":
        lower_limits = np.asarray(problem.get_variable_lower_bounds())
        upper_limits = np.asarray(problem.get_variable_upper_bounds())
        individuals = lhs(problem.n_of_variables, samples=pop_size)
        # Scaling
        individuals = individuals * (upper_limits - lower_limits) + lower_limits

        return individuals

SyntaxError: invalid syntax (<ipython-input-4-12e7fc6b3c7f>, line 1)

In [4]:
#np.sqrt(mean_squared_error(self.y, y_pred))

from sklearn.metrics import mean_squared_error, r2_score

#r2_score(self.y, y_pred)

## Assignment 2

Generate 100 samples (LHS sampling) with the welded beam design problem (K=2, n=4). 

Build two
types of surrogate models for each objective and ﬁnd the R-squared and RMSE for K fold cross
validations. 

Suggest which models to use (for each objective) for performing optimization based on
the accuracies. 

Finally generate 50 more LHS samples and test the models and show their RMSEs


- Use DESDEO, sklearn, GPy
- Optimize the hyperparameters for Kriging
- You can also try to optimize the hyperparameters for random forest

##### what should be done

100 pistettä jakaa Kfold CVllä.. GPR ja Randomforest obj1 ja obj2. Seurata näiden opetus R2 ja RMSE. Sitten optimoida parameterejä kunnes tyytyväinen ja generoida 50 pistettä lisää, joille sama homma. Testata malleja.


TODO: 
 optimize krigin params.. 
 f1 näyttää niin hyvältä että sitä ei varmaan tarvitse optimoida pikemminkin vain jos löytää miksi niin hyvä
 f2 sitä voisi sitten koittaa parantaa. Luulisi että normalisointi auttaiis mutta en tiedä..
 Onko probleema edes formuloitu oikein kun ei ole og paperia missä biobjective formulointi löytynyt?

## Result 

|  |  | Objective 1 K | | Objective 1 RF |  Objective 2 K | Objective 2 RF |
| --- | --- | --- | --- | --- | -- | -- |
| Kfold CV R^2 |  |  |   |   |   | |
| kfold CV RMSE |  |  |  |  |   | |
| Test R^2 |  | |  |   |  | |
| Test RMSE |  |  |  |   |   | |

In [5]:
# the problem, should be correct one

P = 6000
C = 0.000036578 # check if enough zeroes should be 3.6578 times 10^-4

def f1(x):
    return 1.10471*x[:,0]**2*x[:,1] + 0.04811*x[:,2]*x[:,3]*(14 + x[:,1])


def f2(x):
    return (P / (x[:,3]*x[:,2]**3)) * C


The two objectives are the fabrication cost of the beam and the deflection of the end of the beam under the applied load P. The load P is fixed at 6,000 lbs, and the distance L is fixed at 14 in.

The design variables are:

x(1) = h, the thickness of the welds

x(2) = l, the length of the welds

x(3) = t, the height of the beam

x(4) = b, the width of the beam

In [6]:
from sklearn.gaussian_process import GaussianProcessRegressor as GPR

Krigin and Random Forests were suggested.. 

In [7]:
def create_samples(samples):
    # create samples 
    x = lhs(4, samples)
    # could rescale if needed. sklearn GPR assumes that mean is  0 ?
    #x = (x - 0.5)
    return x
    
#func1.shape

### GRP from sklearn 
- TODO : kfold cross validation 

In [8]:
from sklearn.pipeline import Pipeline

from sklearn.gaussian_process.kernels import RBF, ExpSineSquared, WhiteKernel, RationalQuadratic, DotProduct, ConstantKernel, Matern

In [9]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

## CREATE SAMPLES WITH LHS

it really should just be as simple as below ?!
https://scikit-learn.org/stable/modules/cross_validation.html

In [10]:
x = create_samples(100)
func1 = f1(x)
func2 = f2(x)
f = np.vstack((func1, func2)).T

data = pd.DataFrame(np.hstack((x, f)), columns=["h", "l", "t", "b", "f1", "f2"])
f1_mean = func1.mean()
f2_mean = func2.mean()
print(f"Means of objective values with samples f1: {f1_mean} and f2: {f2_mean}")
data[:5]

Means of objective values with samples f1: 0.3439207018890032 and f2: 23235247.22762987


,h,l,t,b,f1,f2
0,0.452819,0.257094,0.404499,0.508666,0.199365,6.519092
1,0.512293,0.584729,0.885964,0.763484,0.644151,0.413355
2,0.586808,0.986071,0.176000,0.433791,0.430146,92.801583
3,0.010268,0.780786,0.778205,0.882939,0.488695,0.527423
4,0.982737,0.344753,0.028916,0.334136,0.374484,27167.851681


In [35]:
from sklearn.gaussian_process import kernels
from sklearn.model_selection import GridSearchCV

seed = 7

parameters = {
    "alpha": [1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4,
              1e-3, 1e-2, 1e-1, 1e-0, 1e1, 1e2, 1e3, 1e4],
    "kernel": [
        kernels.RBF(length_scale_bounds=(1e-10, 1000000)),
        kernels.RationalQuadratic(length_scale_bounds=(1e-10, 1000000)),
        kernels.DotProduct(sigma_0_bounds=(1e-10, 1000000)),
        kernels.Matern(length_scale_bounds=(1e-10, 1000000)),
        kernels.ExpSineSquared(length_scale_bounds=(1e-10, 1000000))
    ]
}

gaussianProcessRegressor_y1 = GridSearchCV(GPR(normalize_y=True, n_restarts_optimizer=1,random_state=seed), parameters)
gaussianProcessRegressor_y2 = GridSearchCV(GPR(normalize_y=True,n_restarts_optimizer=1,random_state=seed), parameters)

# fit best parameters
gaussianProcessRegressor_y1.fit(x, func1)
gaussianProcessRegressor_y1 = gaussianProcessRegressor_y1.best_estimator_
gaussianProcessRegressor_y2.fit(x, func2)
#gaussianProcessRegressor_y2.fit(x[:, 2:], func2) #ignoring the not needed xs
gaussianProcessRegressor_y2 = gaussianProcessRegressor_y2.best_estimator_
print(gaussianProcessRegressor_y1)
print(gaussianProcessRegressor_y2)

GaussianProcessRegressor(alpha=1e-06,
                         kernel=RationalQuadratic(alpha=1, length_scale=1),
                         n_restarts_optimizer=1, normalize_y=True,
                         random_state=7)
GaussianProcessRegressor(alpha=1000.0, kernel=Matern(length_scale=1, nu=1.5),
                         n_restarts_optimizer=1, normalize_y=True,
                         random_state=7)


In [36]:
from sklearn.model_selection import cross_val_score


cross_val_score(gaussianProcessRegressor_y1, x, func1, cv=10, scoring='r2')

array([0.99975493, 0.99918531, 0.99969459, 0.99876425, 0.99862832,
       0.99865644, 0.9994967 , 0.99923207, 0.99749856, 0.99852198])

In [37]:
cross_val_score(gaussianProcessRegressor_y2, x, func2, cv=10, scoring='r2')

array([-1.01179333e+07, -2.19463155e+08, -2.64108031e+13, -9.49435023e+11,
       -1.13137563e+06, -2.35209985e+11, -1.11108375e-01, -6.61267818e+05,
       -5.11149097e+11, -4.71934495e+07])

In [38]:
m, std = gaussianProcessRegressor_y1.predict(x, return_std=True)
m, std

(array([0.1995449 , 0.64411487, 0.43014886, 0.48866712, 0.37448026,
        0.03093177, 0.73144869, 0.01566588, 0.00979845, 0.35097896,
        0.02623518, 0.07645653, 0.22366867, 0.2968595 , 0.13408699,
        0.09092056, 0.46961292, 0.28270871, 0.47696394, 0.64951695,
        0.51773742, 0.32912897, 0.64044376, 0.25525032, 0.64577882,
        0.7803725 , 0.10567733, 0.3957686 , 0.35957662, 0.52932006,
        0.45974794, 0.05017142, 0.59552831, 0.54482963, 0.56287385,
        0.1118312 , 0.34153856, 0.2128833 , 1.04785116, 0.83662389,
        0.16072066, 0.05142841, 0.69557704, 0.50818625, 0.6632105 ,
        0.40831791, 0.31697786, 0.09447876, 0.04600992, 0.40980085,
        0.28127017, 0.64357839, 0.358381  , 0.41567228, 0.32974531,
        0.41086284, 0.10904884, 0.18394405, 0.08214844, 0.58607056,
        0.38209291, 0.46381793, 0.14646258, 0.06277221, 0.18899304,
        0.13428556, 0.3241466 , 0.44428288, 0.34254674, 0.12203093,
        0.11275086, 0.74664905, 0.43470774, 0.61

In [39]:
m, std = gaussianProcessRegressor_y2.predict(x, return_std=True)
m, std

(array([23234931.5661938 , 23234209.55207624, 23235320.51811856,
        23233816.10489636, 23236481.45563307, 23235702.7963833 ,
        23234349.10107119, 23233706.6510722 , 23232459.68923299,
        23233714.90615416, 23235637.81356643, 23233024.48942355,
        23235929.53405555, 23235104.00309398, 23236602.38928251,
        23235081.84447247, 23234861.39400398, 23233658.20374422,
        23234111.75081616, 23234737.79290875, 23234260.33443259,
        23235076.79647023, 23235813.00382766, 23233272.47354869,
        23233067.24455168, 23234313.41817789, 23234876.28719698,
        23233180.0123767 , 23234819.24625159, 23233957.0764161 ,
        23232908.89537481, 23234770.27673857, 23236342.26162861,
        23236128.15424662, 23234555.72530779, 23237148.93631208,
        23236176.17371664, 23233552.89884736, 23235653.58440166,
        23234586.99830491, 23231861.57319217, 23234377.67994055,
        23235702.70602736, 23233354.69779154, 23235170.10992999,
        23235628.53041637

#### from the data 
as one can see x ranges from [0,1] bc LHS. F1 seems to get values with mean of about 0.35 and f2 with way bigger numbers. So should one normalize f2 separetely or to do what with this big difference ?

## F1 with GPR

In [231]:
from sklearn.model_selection import cross_val_score

gpr = GPR()
gpr.fit(x,func1)

# this is jsut using cross val score. But need to do the splitting? 
scores = cross_val_score(gpr, x, func1, cv=5, scoring='r2')
print(scores)

m, std = gpr.predict(x, return_std=True)
print(mean_squared_error(m,func1, squared=False)) # dunno if this correct ?
# print(r2_score(mean, func1))

[0.9996066  0.99985174 0.99895904 0.99975568 0.99992012]
7.926660062146229e-08


## F1 with RF

In [232]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(random_state=7)
regr.fit(x, func1)
scores = cross_val_score(regr, x, func1, cv=5, scoring='r2')
print(scores)
rm = regr.predict(x)
print(mean_squared_error(rm,func1, squared=False))
#print(r2_score(rm, func1))

[0.9304012  0.87473371 0.74688544 0.86391013 0.92384354]
0.03684535544060995


## F2 with GPR

In [233]:
gpr2 = GPR()
#gpr2 = GPR(normalize_y=True)
gpr2.fit(x,func2)

# this is jsut using cross val score. But need to do the splitting? 
scores2 = cross_val_score(gpr2, x, func2, cv=5, scoring='r2')
print(scores2)

m2, std = gpr2.predict(x, return_std=True)
print(mean_squared_error(m2,func2, squared=False)) # dunno if this correct ?

[-1.56805751e+10 -1.16227983e+05 -4.03055748e+05 -5.39012560e-02
 -2.57163880e+07]
53.20269192003474


## F2 with RF

In [234]:
regr2 = RandomForestRegressor(random_state=7)
regr2.fit(x, func2)

rscores2 = cross_val_score(regr2, x, func2, cv=5, scoring='r2')
print(rscores2)

rm2 = regr2.predict(x)
print(mean_squared_error(rm2,func2, squared=False))
#print(r2_score(rm2, func2))

[-1.54371871e+00 -4.88578321e+03 -1.32843025e+03 -4.84898502e-02
  1.62984707e-01]
869825.7256647566


## Get test samples

In [40]:
# gen the test data

x_test = create_samples(50)
func_test1 = f1(x_test)
func_test2 = f2(x_test)
f1t_mean = func_test1.mean()
f2t_mean = func_test2.mean()
print(f"Means of test objective values with test samples f1: {f1t_mean} and f2: {f2t_mean}")
f_test = np.vstack((func_test1, func_test2)).T
test_data = pd.DataFrame(np.hstack((x_test, f_test)), columns=["h", "l", "t", "b", "f1", "f2"])
test_data[:5]

Means of test objective values with test samples f1: 0.383279562377923 and f2: 329222585.8429057


,h,l,t,b,f1,f2
0,0.563538,0.208288,0.264922,0.324709,0.131875,36.351602
1,0.655374,0.495457,0.868391,0.343040,0.442833,0.976970
2,0.128710,0.224082,0.699472,0.890044,0.430132,0.720525
3,0.421948,0.345727,0.143778,0.433464,0.111012,170.350467
4,0.778437,0.689512,0.849889,0.360994,0.678392,0.990342


In [42]:
mean, std = gaussianProcessRegressor_y1.predict(x_test, return_std=True)
print(mean_squared_error(mean,func_test1, squared=False)) # dunno if this correct ?
print(r2_score(mean, func_test1))

0.0036963495083106933
0.9997958910881756


In [41]:
mean, std = gaussianProcessRegressor_y2.predict(x_test, return_std=True)
print(mean_squared_error(mean,func_test2, squared=False)) # dunno if this correct ?
print(r2_score(mean, func_test2))

2324774192.6438375
-5989142724818.657


## GPR F1 TEST

In [236]:
mean, std = gpr.predict(x_test, return_std=True)
print(mean_squared_error(mean,func_test1, squared=False)) # dunno if this correct ?
print(r2_score(mean, func_test1))

0.0028721956277529335
0.999881178980473


## RF F1 TEST

In [237]:
rmean = regr.predict(x_test)
print(mean_squared_error(rmean,func_test1, squared=False)) # dunno if this correct ?
print(r2_score(rmean, func_test1))

0.0896142328508594
0.8131621493260025


## GPR F2 TEST

In [238]:
mean2, std2 = gpr2.predict(x_test,return_std=True)
print(mean_squared_error(mean2,func_test2, squared=False))
print(r2_score(mean2, func_test2))

5667383.045929776
-0.001482976046963902


## RF F2 TEST

In [239]:
rmean2 = regr2.predict(x_test)
print(mean_squared_error(rmean2,func_test2, squared=False))
print(r2_score(rmean2, func_test2))

466451.9728315159
0.15984132791606342


## TODO:

to me the above looks correctly done BUT gpr1 seems to be too good and gpr2 seems to be too bad?

if f2 has smaller mean that test_f2, our r2 goes negative. DUnno what to do with this


Atleast random forests are consistent with the issue. Even though wont be as good for f1, still bad for f2 and in the same way.

RF seems to work better f2, especially if train and test set has similar magnitude mean. This sounds like I should someway properly do the normalization.

In [84]:

## sklearn, with func1
#gpr = GPR()
#gpr.fit(x,func1)
#mean, std = gpr.predict(x, return_std=True)
#print(mean_squared_error(mean,func1)) # dunno if this correct ?
#print(r2_score(mean, func1))

In [86]:
## sklearn, with func2
#gpr = GPR()
#gpr.fit(x,func2)
#mean, std = gpr.predict(x, return_std=True)
#print(mean_squared_error(mean,func2))
#print(r2_score(mean, func2))

### RF from sklearn 
- TODO : kfold cross validation 

In [26]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(random_state=7)
regr.fit(x, func1)
m = regr.predict(x)
print(mean_squared_error(m,func1))
print(r2_score(m, func1))

0.0015696025359983495
0.9764521506416795


In [25]:
regr = RandomForestRegressor(random_state=7)
regr.fit(x, func2)
m = regr.predict(x)
print(mean_squared_error(m,func2))
print(r2_score(m, func2))

42353013631.449585
0.5427258801350856


 ## trying with desdeo

In [92]:
problem = DataProblem(data=data, variable_names=["h", "l", "t", "b"], objective_names=["f1", "f2"])

In [93]:
from desdeo_problem.surrogatemodels.SurrogateModels import GaussianProcessRegressor

In [57]:
#
# Desdeo styl
y = problem.train(gpr)
#problem.train(GaussianProcessRegressor)
#gpr = GPR()

y = problem.train(GPR)
y
#gpr.score(x,y)